In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-cased", num_labels=1)
model.load_state_dict(torch.load("../models/hiring_filter.pth"))
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
test_df = pd.read_csv("../data/raw/hiring/test.csv", index_col=None)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
def tokenize(example):
    return tokenizer(example["prompt"], padding="max_length", truncation=True, max_length=512)

In [5]:
test_tokenized = test_dataset.map(tokenize, batched=True)
test_tokenized = test_tokenized.remove_columns(["prompt", "category"])
test_tokenized = test_tokenized.rename_column("violates_policy", "labels")
test_tokenized.set_format("torch")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
test_dataloader = DataLoader(test_tokenized, batch_size=16)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
probability_scores = []
labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits.squeeze()
        probabilities = torch.sigmoid(logits).detach().cpu().numpy()
        probability_scores.extend(probabilities)
        labels.extend(batch["labels"].detach().cpu().numpy())

In [9]:
accuracy = accuracy_score(labels, np.array(probability_scores) > 0.5)
auroc = roc_auc_score(labels, probability_scores)
f1 = f1_score(labels, np.array(probability_scores) > 0.5)
conf_matrix = confusion_matrix(labels, np.array(probability_scores) > 0.5)

print("Accuracy:", accuracy)
print("AUROC:", auroc)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.86
AUROC: 0.9496
F1 Score: 0.8409090909090909
Confusion Matrix:
 [[49  1]
 [13 37]]
